In [1]:
# 구글 드라이브 연결

from google.colab import drive
import os

drive.mount('/content/drive')

project_path = '/content/drive/MyDrive/ch6'
os.makedirs(project_path, exist_ok=True)

# 현재 경로를 /content/drive로 설정
os.chdir(project_path)
print("현재 작업 디렉토리:", os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
현재 작업 디렉토리: /content/drive/MyDrive/ch6


In [5]:
!apt-get update && apt-get install -y gcc python3.10-venv
!gcc -fno-stack-protector -z execstack -no-pie -o bof bof_taint.c
!ls bof

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [7]:
!python3 -m venv manticore_env
!source manticore_env/bin/activate

In [ ]:
!pip install protobuf==3.19.5 z3-solver==4.13.0.0 jedi>=0.16
!pip install manticore

bof


In [9]:
!nm -C ./bof | grep "vulnerable_function"

00000000004013c1 T vulnerable_function


In [3]:
from manticore import Manticore

# 분석할 바이너리 파일 지정
binary_path = './bof'  # 실제 파일 경로로 수정

# Manticore 인스턴스 생성
m = Manticore(binary_path)

# Taint Analysis를 위한 콜백 설정
@m.hook(0x4013c1)  # 이 주소는 바이너리에서 추출한 후 적절히 수정
def taint_analysis(state):
    # Taint 입력값 추적
    stdin = state.posix.stdin
    tainted_data = stdin.read(32)  # 32바이트의 입력을 Taint 처리
    state.taint(tainted_data, "user_input")

    print(f"Input is tainted: {tainted_data}")

    # 특정 메모리 위치나 레지스터에서 Taint 상태 확인
    if state.is_tainted(tainted_data):
        print(f"Tainted data found at memory address: {hex(state.cpu.RIP)}")

# 분석 시작
m.run()

# 분석 결과 출력
for input_data in m.context['tainted_data']:
    print(f"Tainted input: {input_data}")


ImportError: cannot import name 'Manticore' from 'manticore' (/usr/local/lib/python3.10/dist-packages/manticore/__init__.py)

In [ ]:
!echo "AZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZsecret123ddd" | ./bof

Enter the input (type 'quit' to exit): Modifying input...
Input after modification: AZZZXZZZZZZZZZZZZZZZZZZZZZZZZZZZsecret123ddd
All conditions met! Executing vulnerable function...
Buffer contents: AZZZXZZZZZZZZZZZZZZZZZZZZZZZZZZZsecret123ddd
/bin/bash: line 1: 26226 Done                    echo "AZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZsecret123ddd"
     26227 Segmentation fault      (core dumped) | ./bof


In [ ]:
from triton import TritonContext, ARCH, Instruction, MemoryAccess

# TritonContext 생성 및 아키텍처 설정
ctx = TritonContext()
ctx.setArchitecture(ARCH.X86_64)

# 바이너리 파일 로드 및 설정
binary_path = "./program"
ctx.loadBinary(binary_path)

# 분석할 메모리 주소와 레지스터 설정
vulnerable_func_addr = 0x4013c1  # gdb에서 추출한 vulnerable_function의 주소

# 분석 시작 주소 설정
start_address = ctx.getEntryPoint()
print(f"Starting analysis at address: {hex(start_address)}")

# 초기 레지스터 값 설정
ctx.setConcreteRegisterValue(ctx.registers.rdi, 0x7fffffffe000)  # 예시로 RDI 레지스터 값 설정
ctx.taintMemory(0x7fffffffe000)  # 입력값이 저장될 메모리 위치를 taint 소스로 설정

# 명령어 분석
while ctx.isRunning():
    # 현재 주소에서 명령어 추출 및 실행
    instruction = ctx.fetchInstruction()
    ctx.processing(instruction)

    # 특정 함수에 도달했는지 확인 (vulnerable_function)
    if instruction.getAddress() == vulnerable_func_addr:
        print(f"[+] Reached vulnerable_function at address: {hex(vulnerable_func_addr)}")
        break

    # 메모리와 레지스터의 taint 상태 출력
    if ctx.isRegisterTainted(ctx.registers.rax):
        print(f"RAX 레지스터가 tainted 상태입니다.")
    if ctx.isMemoryTainted(0x7fffffffe000):
        print(f"메모리 0x7fffffffe000이 tainted 상태입니다.")


ModuleNotFoundError: No module named 'triton'